In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('dataset4') 

# Actions that we try to detect
actions = np.array(['Help', 'please', 'sorry'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [8]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [9]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
#                 print(results)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [11]:
label_map = {label:num for num, label in enumerate(actions)}

In [12]:
label_map

{'Help': 0, 'please': 1, 'sorry': 2}

In [13]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [14]:
np.array(sequences).shape

(90, 30, 1662)

In [15]:
np.array(labels).shape

(90,)

In [16]:
X = np.array(sequences)

In [17]:
X.shape

(90, 30, 1662)

In [18]:
y = to_categorical(labels).astype(int)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [20]:
y_test.shape

(5, 3)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard


In [22]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [23]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [24]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [25]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 7s 992ms/step - loss: 1.9084 - categorical_accuracy: 0.3176
Epoch 2/2000
3/3 [==============================] - ETA: 0s - loss: 3.4686 - categorical_accuracy: 0.29 - 1s 222ms/step - loss: 3.4686 - categorical_accuracy: 0.2941
Epoch 3/2000
3/3 [==============================] - 0s 159ms/step - loss: 1.1094 - categorical_accuracy: 0.3294
Epoch 4/2000
3/3 [==============================] - 0s 62ms/step - loss: 1.1672 - categorical_accuracy: 0.3529
Epoch 5/2000
3/3 [==============================] - 0s 65ms/step - loss: 6.7665 - categorical_accuracy: 0.3529
Epoch 6/2000
3/3 [==============================] - 1s 229ms/step - loss: 2.5123 - categorical_accuracy: 0.4000
Epoch 7/2000
3/3 [==============================] - 0s 83ms/step - loss: 1.1247 - categorical_accuracy: 0.2941
Epoch 8/2000
3/3 [==============================] - 0s 64ms/step - loss: 2.2554 - categorical_accuracy: 0.3529
Epoch 9/2000
3/3 [==============================] - 0s

3/3 [==============================] - 1s 447ms/step - loss: 1.5440 - categorical_accuracy: 0.2941
Epoch 73/2000
3/3 [==============================] - 0s 153ms/step - loss: 1.5968 - categorical_accuracy: 0.2824
Epoch 74/2000
3/3 [==============================] - 1s 160ms/step - loss: 1.4905 - categorical_accuracy: 0.3529
Epoch 75/2000
3/3 [==============================] - 1s 163ms/step - loss: 1.4516 - categorical_accuracy: 0.2824
Epoch 76/2000
3/3 [==============================] - 0s 105ms/step - loss: 2.4963 - categorical_accuracy: 0.3294
Epoch 77/2000
3/3 [==============================] - 0s 147ms/step - loss: 3.9204 - categorical_accuracy: 0.3882
Epoch 78/2000
3/3 [==============================] - 0s 117ms/step - loss: 1.2686 - categorical_accuracy: 0.4471
Epoch 79/2000
3/3 [==============================] - 1s 419ms/step - loss: 1.4655 - categorical_accuracy: 0.3647
Epoch 80/2000
3/3 [==============================] - 0s 126ms/step - loss: 3.7188 - categorical_accuracy: 0.44

3/3 [==============================] - 1s 370ms/step - loss: 0.9692 - categorical_accuracy: 0.5765
Epoch 144/2000
3/3 [==============================] - 0s 107ms/step - loss: 0.9354 - categorical_accuracy: 0.6941
Epoch 145/2000
3/3 [==============================] - 0s 155ms/step - loss: 0.8832 - categorical_accuracy: 0.6706
Epoch 146/2000
3/3 [==============================] - 1s 460ms/step - loss: 0.8567 - categorical_accuracy: 0.7882
Epoch 147/2000
3/3 [==============================] - 0s 146ms/step - loss: 0.8119 - categorical_accuracy: 0.6471
Epoch 148/2000
3/3 [==============================] - 1s 188ms/step - loss: 0.8570 - categorical_accuracy: 0.5647
Epoch 149/2000
3/3 [==============================] - 0s 164ms/step - loss: 0.7301 - categorical_accuracy: 0.6941
Epoch 150/2000
3/3 [==============================] - 0s 85ms/step - loss: 0.7996 - categorical_accuracy: 0.5765
Epoch 151/2000
3/3 [==============================] - 1s 418ms/step - loss: 0.7872 - categorical_accurac

3/3 [==============================] - 0s 184ms/step - loss: 0.5083 - categorical_accuracy: 0.7647
Epoch 216/2000
3/3 [==============================] - 0s 148ms/step - loss: 0.3927 - categorical_accuracy: 0.8353
Epoch 217/2000
3/3 [==============================] - 1s 284ms/step - loss: 0.4096 - categorical_accuracy: 0.7765
Epoch 218/2000
3/3 [==============================] - 0s 135ms/step - loss: 0.4231 - categorical_accuracy: 0.7529
Epoch 219/2000
3/3 [==============================] - 0s 168ms/step - loss: 0.3969 - categorical_accuracy: 0.7176
Epoch 220/2000
3/3 [==============================] - 0s 124ms/step - loss: 0.4067 - categorical_accuracy: 0.7647
Epoch 221/2000
3/3 [==============================] - 0s 156ms/step - loss: 0.4200 - categorical_accuracy: 0.8000
Epoch 222/2000
3/3 [==============================] - 1s 198ms/step - loss: 0.4682 - categorical_accuracy: 0.6706
Epoch 223/2000
3/3 [==============================] - 0s 158ms/step - loss: 0.3684 - categorical_accura

3/3 [==============================] - 0s 187ms/step - loss: 0.5774 - categorical_accuracy: 0.6941
Epoch 286/2000
3/3 [==============================] - 0s 186ms/step - loss: 0.7138 - categorical_accuracy: 0.6706
Epoch 287/2000
3/3 [==============================] - 0s 145ms/step - loss: 0.7031 - categorical_accuracy: 0.7294
Epoch 288/2000
3/3 [==============================] - 0s 154ms/step - loss: 0.6330 - categorical_accuracy: 0.6588
Epoch 289/2000
3/3 [==============================] - 1s 299ms/step - loss: 0.6998 - categorical_accuracy: 0.5882
Epoch 290/2000
3/3 [==============================] - 0s 177ms/step - loss: 1.0272 - categorical_accuracy: 0.3882
Epoch 291/2000
3/3 [==============================] - 0s 122ms/step - loss: 0.7703 - categorical_accuracy: 0.5529
Epoch 292/2000
3/3 [==============================] - 1s 302ms/step - loss: 0.8199 - categorical_accuracy: 0.6588
Epoch 293/2000
3/3 [==============================] - 1s 133ms/step - loss: 0.7743 - categorical_accura

...

3/3 [==============================] - 1s 220ms/step - loss: 1.3197e-06 - categorical_accuracy: 1.0000
Epoch 1662/2000
3/3 [==============================] - 1s 176ms/step - loss: 1.3169e-06 - categorical_accuracy: 1.0000
Epoch 1663/2000
3/3 [==============================] - 1s 227ms/step - loss: 1.3155e-06 - categorical_accuracy: 1.0000
Epoch 1664/2000
3/3 [==============================] - 1s 165ms/step - loss: 1.3127e-06 - categorical_accuracy: 1.0000
Epoch 1665/2000
3/3 [==============================] - 1s 354ms/step - loss: 1.3127e-06 - categorical_accuracy: 1.0000
Epoch 1666/2000
3/3 [==============================] - 1s 235ms/step - loss: 1.3113e-06 - categorical_accuracy: 1.0000
Epoch 1667/2000
3/3 [==============================] - 1s 269ms/step - loss: 1.3113e-06 - categorical_accuracy: 1.0000
Epoch 1668/2000
3/3 [==============================] - 0s 141ms/step - loss: 1.3071e-06 - categorical_accuracy: 1.0000
Epoch 1669/2000
3/3 [==============================] - 1s 149ms/

3/3 [==============================] - 1s 181ms/step - loss: 1.1809e-06 - categorical_accuracy: 1.0000
Epoch 1730/2000
3/3 [==============================] - 1s 182ms/step - loss: 1.1781e-06 - categorical_accuracy: 1.0000
Epoch 1731/2000
3/3 [==============================] - 1s 229ms/step - loss: 1.1739e-06 - categorical_accuracy: 1.0000
Epoch 1732/2000
3/3 [==============================] - 1s 385ms/step - loss: 1.1682e-06 - categorical_accuracy: 1.0000
Epoch 1733/2000
3/3 [==============================] - 1s 241ms/step - loss: 1.1654e-06 - categorical_accuracy: 1.0000
Epoch 1734/2000
3/3 [==============================] - 1s 258ms/step - loss: 1.1640e-06 - categorical_accuracy: 1.0000
Epoch 1735/2000
3/3 [==============================] - 1s 460ms/step - loss: 1.1640e-06 - categorical_accuracy: 1.0000
Epoch 1736/2000
3/3 [==============================] - 1s 231ms/step - loss: 1.1626e-06 - categorical_accuracy: 1.0000
Epoch 1737/2000
3/3 [==============================] - 2s 554ms/

Epoch 1798/2000
3/3 [==============================] - 1s 422ms/step - loss: 1.0434e-06 - categorical_accuracy: 1.0000
Epoch 1799/2000
3/3 [==============================] - 1s 279ms/step - loss: 1.0434e-06 - categorical_accuracy: 1.0000
Epoch 1800/2000
3/3 [==============================] - 1s 604ms/step - loss: 1.0420e-06 - categorical_accuracy: 1.0000
Epoch 1801/2000
3/3 [==============================] - 2s 494ms/step - loss: 1.0420e-06 - categorical_accuracy: 1.0000
Epoch 1802/2000
3/3 [==============================] - 1s 185ms/step - loss: 1.0406e-06 - categorical_accuracy: 1.0000
Epoch 1803/2000
3/3 [==============================] - 1s 200ms/step - loss: 1.0378e-06 - categorical_accuracy: 1.0000
Epoch 1804/2000
3/3 [==============================] - 1s 153ms/step - loss: 1.0378e-06 - categorical_accuracy: 1.0000
Epoch 1805/2000
3/3 [==============================] - 0s 146ms/step - loss: 1.0336e-06 - categorical_accuracy: 1.0000
Epoch 1806/2000
3/3 [===========================

3/3 [==============================] - 0s 140ms/step - loss: 9.2983e-07 - categorical_accuracy: 1.0000
Epoch 1867/2000
3/3 [==============================] - 0s 127ms/step - loss: 9.2983e-07 - categorical_accuracy: 1.0000
Epoch 1868/2000
3/3 [==============================] - 1s 169ms/step - loss: 9.2843e-07 - categorical_accuracy: 1.0000
Epoch 1869/2000
3/3 [==============================] - 1s 241ms/step - loss: 9.2843e-07 - categorical_accuracy: 1.0000
Epoch 1870/2000
3/3 [==============================] - 0s 146ms/step - loss: 9.2562e-07 - categorical_accuracy: 1.0000
Epoch 1871/2000
3/3 [==============================] - 0s 163ms/step - loss: 9.2562e-07 - categorical_accuracy: 1.0000
Epoch 1872/2000
3/3 [==============================] - 1s 188ms/step - loss: 9.2282e-07 - categorical_accuracy: 1.0000
Epoch 1873/2000
3/3 [==============================] - 1s 119ms/step - loss: 9.2282e-07 - categorical_accuracy: 1.0000
Epoch 1874/2000
3/3 [==============================] - 0s 128ms/

3/3 [==============================] - 1s 172ms/step - loss: 8.2745e-07 - categorical_accuracy: 1.0000
Epoch 1935/2000
3/3 [==============================] - 0s 113ms/step - loss: 8.2465e-07 - categorical_accuracy: 1.0000
Epoch 1936/2000
3/3 [==============================] - 1s 314ms/step - loss: 8.2184e-07 - categorical_accuracy: 1.0000
Epoch 1937/2000
3/3 [==============================] - 1s 270ms/step - loss: 8.2184e-07 - categorical_accuracy: 1.0000
Epoch 1938/2000
3/3 [==============================] - 1s 462ms/step - loss: 8.2184e-07 - categorical_accuracy: 1.0000
Epoch 1939/2000
3/3 [==============================] - 1s 190ms/step - loss: 8.2044e-07 - categorical_accuracy: 1.0000
Epoch 1940/2000
3/3 [==============================] - 1s 221ms/step - loss: 8.1904e-07 - categorical_accuracy: 1.0000
Epoch 1941/2000
3/3 [==============================] - 0s 135ms/step - loss: 8.1623e-07 - categorical_accuracy: 1.0000
Epoch 1942/2000
3/3 [==============================] - 1s 230ms/

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 596,675
Trainable params: 596,675
Non-trainable params: 0
__________________________________________________

In [27]:
res = model.predict(X_test)

In [28]:
actions[np.argmax(res[4])]

'please'

In [29]:
actions[np.argmax(y_test[4])]

'Help'

In [30]:
model.save('mod4.h5')

In [31]:
model.load_weights('mod4.h5')

In [32]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [33]:

yhat = model.predict(X_test)

In [34]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [35]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 0],
        [1, 1]],

       [[3, 1],
        [0, 1]],

       [[3, 0],
        [0, 2]]], dtype=int64)

In [36]:
accuracy_score(ytrue, yhat)

0.8

In [37]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [39]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.S

please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
please
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Help
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Help
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Help
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Help
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Help
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Help
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Help
<class 'mediapipe.python.solution_base.SolutionOutputs'>
Help
<class 'mediapipe.python.solution_base.SolutionOu